# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [13]:
from azureml.core import Workspace, Experiment, Model
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Environment, ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
from azureml.train.hyperdrive.parameter_expressions import choice
import joblib

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [14]:
ws = Workspace.from_config()
experiment_name = 'my-experiment'

experiment=Experiment(ws, experiment_name)

In [15]:
cpu_cluster_name = "hyper-drive"
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Cluster Found!.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    cpu_cluster.wait_for_completion(show_output=True)

Cluster Found!.


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [16]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
                 "--C" : uniform(0.1,1),
                 "--max_iter" : choice(50,100,150,200) })

#TODO: Create your estimator and hyperdrive config
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')
src = ScriptRunConfig(source_directory = '.', compute_target=cpu_cluster_name, script='train.py', environment=sklearn_env)

hyperdrive_run_config = HyperDriveConfig(run_config=src,
                                hyperparameter_sampling=param_sampling,
                                policy = early_termination_policy,
                                primary_metric_name="Accuracy",
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=10,
                                max_concurrent_runs=4)

# ## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [17]:
#TODO: Submit your experiment

hyperdrive_run = experiment.submit(hyperdrive_run_config,show_output=True)



In [18]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_edcaeb57-6a54-4ac5-93c8-787df5de8ad0
Web View: https://ml.azure.com/runs/HD_edcaeb57-6a54-4ac5-93c8-787df5de8ad0?wsid=/subscriptions/f9d5a085-54dc-4215-9ba6-dad5d86e60a0/resourcegroups/aml-quickstarts-239402/workspaces/quick-starts-ws-239402&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-07-29T15:46:59.885363][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-07-29T15:47:00.4050925Z][SCHEDULER][INFO]Scheduling job, id='HD_edcaeb57-6a54-4ac5-93c8-787df5de8ad0_0' 
[2023-07-29T15:47:00.5129325Z][SCHEDULER][INFO]Scheduling job, id='HD_edcaeb57-6a54-4ac5-93c8-787df5de8ad0_1' 
[2023-07-29T15:47:00.6501571Z][SCHEDULER][INFO]Scheduling job, id='HD_edcaeb57-6a54-4ac5-93c8-787df5de8ad0_2' 
[2023-07-29T15:47:00.7995366Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_edcaeb57-6a54-4ac5-93c8-787df5de8ad0_0' 
[2023-07-29T15:47:00.7636329Z][SCHEDULER][INFO]Scheduling job, id='HD_edcaeb57-6a54-4ac5-93c8-787df5de8ad0_3

{'runId': 'HD_edcaeb57-6a54-4ac5-93c8-787df5de8ad0',
 'target': 'hyper-drive',
 'status': 'Completed',
 'startTimeUtc': '2023-07-29T15:46:59.067442Z',
 'endTimeUtc': '2023-07-29T16:01:02.235045Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '2a2afe89-d17c-4016-8769-7db0d8942a05',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': 'infinite_space_size',
  'score': '0.7519685039370079',
  'best_child_run_id': 'HD_edcaeb57-6a54-4ac5-93c8-787df5de8ad0_5',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_edcaeb57-6a54-4ac5-93c8-787df5de8ad0_5'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValu

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [19]:

# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print('best run',best_run)
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']
print('Best Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n learning rate:',parameter_values)

best run Run(Experiment: my-experiment,
Id: HD_edcaeb57-6a54-4ac5-93c8-787df5de8ad0_5,
Type: azureml.scriptrun,
Status: Completed)
Best Id:  HD_edcaeb57-6a54-4ac5-93c8-787df5de8ad0_5

 Accuracy: 0.7519685039370079

 learning rate: ['--C', '0.6814512557436373', '--max_iter', '100']


In [20]:
best_run.get_file_names()

['logs/azureml/dataprep/0/backgroundProcess.log',
 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/0/rslex.log.2023-07-29-15',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

In [21]:
#TODO: Save the best model
joblib.dump(parameter_values, filename='best-hyperdrive.pkl')

['best-hyperdrive.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [22]:
model = Model.register(workspace=ws,model_name = "best-hyperdrive", model_path = './best-hyperdrive.pkl')
print(model.name, model.id, model.version, sep='\t')

Registering model best-hyperdrive
best-hyperdrive	best-hyperdrive:1	1


TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service